In [1]:
import numpy as np 
import pandas as pd
import seaborn as sns 
import nltk
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.preprocessing import LabelBinarizer
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
from wordcloud import WordCloud,STOPWORDS
from nltk.stem import WordNetLemmatizer
from nltk import LancasterStemmer
from nltk.tokenize import word_tokenize,sent_tokenize 
import spacy 
import re,string,unicodedata
from nltk.tokenize.toktok import ToktokTokenizer
from sklearn.linear_model import LinearRegression, SGDClassifier
from sklearn.naive_bayes import MultinomialNB
from sklearn.svm import SVC
from textblob import TextBlob,Word
from sklearn.metrics import classification_report, confusion_matrix,accuracy_score
from bs4 import BeautifulSoup
from nltk.corpus import wordnet
import matplotlib.pyplot as plt


/usr/local/lib/python3.8/dist-packages/torch/cuda/__init__.py:497: UserWarning: Can't initialize NVML
  warnings.warn("Can't initialize NVML")


In [2]:
data = pd.read_csv("/content/IMDB Dataset.csv",encoding='utf-8')
nltk.download("stopwords")
nltk.download('punkt')
nltk.download('wordnet')
nltk.download('averaged_perceptron_tagger')
tokenizers=ToktokTokenizer()
stopwords=nltk.corpus.stopwords.words('english')
nltk.download('omw-1.4')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package omw-1.4 to /root/nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!


True

In [3]:
def noiseremoval_text(text):
    clean = re.compile('<.*?>')
    return re.sub(clean, '', text)
data["review"] = data["review"].apply(noiseremoval_text)
def get_wordnet_pos(treebank_tag):
    # Map POS tag to WordNet POS tag
    if treebank_tag.startswith('J'):
        return wordnet.ADJ
    elif treebank_tag.startswith('V'):
        if treebank_tag == 'VBD':  # Handle past tense verbs
            return 'v'
        else:
            return wordnet.VERB
    elif treebank_tag.startswith('N'):
        return wordnet.NOUN
    elif treebank_tag.startswith('R'):
        return wordnet.ADV
    else:
        return None
def lemmatize_sentence(sentence):
    # Tokenize the sentence into words
    words = nltk.word_tokenize(sentence)

    # POS tag the words
    pos_tags = nltk.pos_tag(words)

    # Initialize the WordNet lemmatizer
    lemmatizer = WordNetLemmatizer()

    # Lemmatize each word in the sentence while preserving the POS tags
    lemmatized_words = []
    for word, pos in pos_tags:
        wordnet_pos = get_wordnet_pos(pos) or wordnet.NOUN
        lemmatized_word = lemmatizer.lemmatize(word, pos=wordnet_pos)
        lemmatized_words.append(lemmatized_word)

    # Join the lemmatized words back into a string
    lemmatized_sentence = ' '.join(lemmatized_words)
    return lemmatized_sentence
data['review']= data['review'].apply(lemmatize_sentence)

In [4]:
data.head()

,review,sentiment
0,One of the other reviewer have mention that af...,positive
1,A wonderful little production . The filming te...,positive
2,I think this be a wonderful way to spend time ...,positive
3,Basically there 's a family where a little boy...,negative
4,Petter Mattei 's `` Love in the Time of Money ...,positive


In [5]:
# removing stopwords
def removing_stopwords(text, is_lower_case=False):
  tokenizers=ToktokTokenizer()
  tokens= tokenizers.tokenize(text)
  tokens= [i.strip() for i in tokens]
  if is_lower_case:
    filtokens= [i for i in tokens if token not in stopwords]
  else:
    filtokens= [i for i in tokens if i.lower() not in stopwords]
  filtered_texts=' '.join(filtokens)
  return filtered_texts

In [6]:
data["review"]=data["review"].apply(removing_stopwords)

In [7]:
data.head()

,review,sentiment
0,One reviewer mention watch 1 Oz episode ' hook...,positive
1,wonderful little production . filming techniqu...,positive
2,think wonderful way spend time hot summer week...,positive
3,Basically ' family little boy ( Jake ) think '...,negative
4,Petter Mattei ' ` ` Love Time Money ' ' visual...,positive


In [17]:
train_reviews_data=data.review[:30000]

In [18]:
test_reviews_data=data.review[30000:]

In [27]:
cv= CountVectorizer(min_df=0,max_df=1, binary=False,ngram_range=(1,3))
cv_train=cv.fit_transform(train_reviews_data)
cv_test=cv.transform(test_reviews_data)
print(cv_train.shape)
print(cv_test.shape)
#burada sözcük sayıları sayılarak vektör oluşturuluyor

(30000, 4702962)
(20000, 4702962)


In [35]:
tf=TfidfVectorizer(min_df=0,max_df=1,use_idf=True,ngram_range=(1,3))
tf_train= tf.fit_transform(train_reviews_data)
tf_test= tf.transform(test_reviews_data)
print(tf_train.shape)
print(tf_test.shape)

(30000, 4702962)
(20000, 4702962)


In [21]:
label=LabelBinarizer()
sentiment_data=label.fit_transform(data['sentiment'])
print(sentiment_data.shape)
#0 ve 1 yapıyor

(50000, 1)


In [29]:
train_data=data.sentiment[:30000]
test_data=data.sentiment[30000:]


In [30]:
from sklearn.linear_model import LogisticRegression

logistic=LogisticRegression(penalty='l2',max_iter=500,C=1,random_state=42)
lr_bow=logistic.fit(cv_train,train_data)
print(lr_bow)
lr_tfidf=logistic.fit(tf_train,train_data)
print(lr_tfidf)

LogisticRegression(C=1, max_iter=500, random_state=42)
LogisticRegression(C=1, max_iter=500, random_state=42)


In [31]:
lr_bow_predict=logistic.predict(cv_test)
print(lr_bow_predict)

['negative' 'negative' 'positive' ... 'negative' 'negative' 'positive']


In [38]:
lr_tfidf_predict=logistic.predict(tf_test)
print(lr_tfidf)

['negative' 'negative' 'positive' ... 'negative' 'negative' 'positive']


In [33]:
lr_bow_score=accuracy_score(test_data,lr_bow_predict)
print(lr_bow_score)

0.74045


In [40]:
lr_tfidf_score=accuracy_score(test_data,lr_tfidf_predict)
print(lr_tfidf_score)

0.74105
